In [ ]:
# 1) Environment detection and summary (safe for local/Colab)
import os, sys, platform, json, datetime
IN_COLAB = 'COLAB_GPU' in os.environ or 'google.colab' in sys.modules
CUDA = False
MPS = False
TPU = False
try:
    import torch
    CUDA = torch.cuda.is_available()
    MPS = getattr(torch.backends, 'mps', None) and torch.backends.mps.is_available()
except Exception:
    pass
try:
    import torch_xla.core.xla_model as xm  # noqa
    TPU = True
except Exception:
    pass
print(json.dumps({
    'in_colab': bool(IN_COLAB),
    'cuda': bool(CUDA),
    'mps': bool(MPS),
    'tpu': bool(TPU),
    'python': sys.version.split()[0],
    'platform': platform.platform(),
    'timestamp': datetime.datetime.utcnow().isoformat()+'Z'
}, indent=2))

# Restoria Colab Notebook

A Colab-optimized walkthrough for using the Restoria toolkit (GFPGAN-compatible).

In [ ]:
# Utility imports used throughout the notebook
from PIL import Image  # noqa: F401


# GFPGAN Colab Demo (Fork)

This Colab notebook installs dependencies, clones this fork, downloads pretrained weights automatically, and runs the GFPGAN inference script on sample images.

In [ ]:
#@title Install dependencies
import os
import sys
import shutil
import subprocess


def run(cmd, shell=False, check=True):
    print(">>", cmd if shell else " ".join(cmd))
    if shell:
        return subprocess.run(cmd, shell=True, check=check)
    else:
        return subprocess.run(cmd, check=check)

print(sys.version)

# CI smoke: robust env var parsing (treat '0'/'false' as False)
_ci_val = os.environ.get("NB_CI_SMOKE", "").strip().lower()
CI_SMOKE = _ci_val in ("1", "true", "yes", "y")
if CI_SMOKE:
    print("NB_CI_SMOKE=1: skipping dependency install step")

if not CI_SMOKE:
    # Upgrade pip tooling quietly
    run([sys.executable, "-m", "pip", "install", "--upgrade", "--quiet", "pip", "setuptools", "wheel"])
    # Ensure IPython deps are satisfied to avoid resolver warnings
    run([sys.executable, "-m", "pip", "install", "--quiet", "-U", "ipython", "jedi"])

    # Install torch pinned to repo-compatible versions
    has_nvidia = shutil.which("nvidia-smi") is not None
    torch_spec = "torch==2.1.*"
    vision_spec = "torchvision==0.16.*"
    audio_spec = "torchaudio==2.1.*"
    if has_nvidia:
        print("Detected NVIDIA GPU; installing CUDA wheels (cu121)")
        run([
            sys.executable,
            "-m",
            "pip",
            "install",
            "--quiet",
            torch_spec,
            vision_spec,
            audio_spec,
            "--index-url",
            "https://download.pytorch.org/whl/cu121",
        ])
    else:
        print("No NVIDIA GPU; installing CPU wheels")
        run([sys.executable, "-m", "pip", "install", "--quiet", torch_spec, vision_spec, audio_spec])

    # Core deps: match repo constraints (basicsr <= 1.4.2)
    run(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "--quiet",
            "--no-cache-dir",
            "--upgrade",
            "basicsr<=1.4.2",
            "facexlib",
            "realesrgan",
            "opencv-python",
            "tqdm",
            "numpy",
            "PyYAML",
        ]
    )

In [ ]:
# Clone the repository
REPO_URL = "https://github.com/IAmJonoBo/Restoria.git"
REPO_DIR = "GFPGAN"  # Keep folder name for compatibility with notebook paths

import os
import shutil
import subprocess

if os.path.exists(REPO_DIR):
    print(f"Removing existing {REPO_DIR} directory...")
    shutil.rmtree(REPO_DIR)

print(f"Cloning repository from {REPO_URL}...")
subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)
print("Clone complete.")

In [ ]:
#@title Run inference on sample images
import os
import sys
import subprocess

# CI smoke: skip heavy inference when NB_CI_SMOKE is set
if os.environ.get("NB_CI_SMOKE"):
    print("NB_CI_SMOKE=1: skipping sample inference step")
else:
    repo_dir = "GFPGAN"
    cmd = [sys.executable, "inference_gfpgan.py", "-i", "inputs/whole_imgs", "-o", "results", "-v", "1.4", "-s", "2"]
    print("Running:", " ".join(cmd))
    res = subprocess.run(cmd, cwd=repo_dir, check=False)
    print("Exit code:", res.returncode)
    print("Results written to ./GFPGAN/results")

In [ ]:
# @title Display a few restored images
import glob
import os
from IPython.display import display, Image

# CI smoke: do not try to display images in headless CI
if os.environ.get("NB_CI_SMOKE"):
    print("NB_CI_SMOKE=1: skipping image display")
else:
    base = "GFPGAN/results/restored_imgs"
    imgs = sorted(glob.glob(os.path.join(base, "*")))[:4]
    print(f"Displaying {len(imgs)} images from", base)
    for p in imgs:
        try:
            display(Image(filename=p))
        except Exception:
            print("Could not display:", p)

In [ ]:
# @title Upload your own images and run GFPGAN
# @markdown Upload one or more images; they will be processed with the settings below.
import os
import shutil
import subprocess
import sys

# Guard: if not in Colab, skip gracefully; also skip in CI smoke
IN_COLAB = False
try:
    import google.colab  # type: ignore
    from google.colab import files
    IN_COLAB = True
except Exception:
    pass

if os.environ.get("NB_CI_SMOKE"):
    print("NB_CI_SMOKE=1: skipping upload/inference UI")
else:
    upload_dir = "GFPGAN/uploads"
    os.makedirs(upload_dir, exist_ok=True)

    if IN_COLAB:
        uploaded = files.upload()
        for name, data in uploaded.items():
            with open(name, "wb") as f:
                f.write(data)
            shutil.move(name, os.path.join(upload_dir, os.path.basename(name)))
    else:
        print("Not running in Colab; skipping upload UI.")

    # Inference parameters
    version = "1.4"  # @param ['1', '1.2', '1.3', '1.4']
    scale = 2  # @param {type: 'integer'}
    only_center_face = False  # @param {type: 'boolean'}
    aligned = False  # @param {type: 'boolean'}
    autopilot = False  # @param {type: 'boolean'}
    hardware_aware = True  # @param {type: 'boolean'}
    select_by = "sharpness"  # @param ['sharpness', 'identity']

    print("Running inference on uploads...")
    cmd = [sys.executable, "inference_gfpgan.py", "-i", "uploads", "-o", "results", "-v", str(version), "-s", str(scale)]
    if only_center_face:
        cmd.append("--only_center_face")
    if aligned:
        cmd.append("--aligned")
    if autopilot:
        cmd.append("--auto")
        cmd.extend(["--select-by", select_by])
    if hardware_aware:
        cmd.append("--auto-hw")
    subprocess.run(cmd, cwd="GFPGAN", check=False)
    print("Done. See GFPGAN/results")

Notes:
- GPU acceleration is optional; uncomment the CUDA wheel index in the install cell to use GPU on Colab.
- The inference script will automatically download the GFPGAN v1.4 weights if not present.
- For large batches, consider enabling the Real-ESRGAN background upsampler by keeping default settings (it is auto-disabled on CPU).

In [ ]:
# 2) Dependencies (Colab-friendly; quiet). Skipped if not in Colab to avoid local env churn.
import subprocess
if 'google.colab' in sys.modules:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-U',
                    'huggingface_hub>=0.23',
                    'Pillow>=9'], check=False)
print('Deps ready (or skipped locally).')



In [ ]:
# 3) Determinism: seed and flags for reproducibility
import random
try:
    import numpy as np
except Exception:
    np = None
try:
    import torch
except Exception:
    torch = None

SEED = 123
DETERMINISTIC = True
random.seed(SEED)
if np is not None:
    np.random.seed(SEED)
if torch is not None:
    torch.manual_seed(SEED)
    if DETERMINISTIC:
        try:
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        except Exception:
            pass
print({'seed': SEED, 'deterministic': DETERMINISTIC})

In [ ]:
# 4) Quickstart: run Restoria in dry-run mode on a sample image
import os, sys, subprocess, json
from pathlib import Path

# Ensure in-repo import works if not installed
if 'restoria' not in sys.modules:
    sys.path.insert(0, str(Path.cwd() / 'src'))

# Find a small sample image
candidates = [
    'assets/gfpgan_logo.png',
    'bench/samples/portrait.jpg',
]
img = None
for c in candidates:
    if os.path.exists(c):
        img = c
        break
if img is None:
    os.makedirs('tmp', exist_ok=True)
    img = 'tmp/blank.png'
    open(img,'wb').write(b'\x89PNG\r\n\x1a\n')

out_dir = 'results/dry_demo/notebook_quickstart'
# Use the Python entrypoint to avoid shell specifics in Colab
from restoria.cli.main import main as restoria_main
rc = restoria_main(['run', '--input', img, '--output', out_dir, '--dry-run', '--device', 'auto'])
print('Return code:', rc)
print('Wrote:', out_dir)

In [ ]:
# 5) Outputs summary: show manifest and metrics (short)
import json, os
out_dir = 'results/dry_demo/notebook_quickstart'
manifest_p = os.path.join(out_dir, 'manifest.json')
metrics_p = os.path.join(out_dir, 'metrics.json')
if os.path.exists(manifest_p):
    m = json.load(open(manifest_p))
    print('device:', m.get('device'))
    print('args.device:', m.get('args',{}).get('device'))
if os.path.exists(metrics_p):
    mm = json.load(open(metrics_p))
    recs = mm.get('metrics', [])
    print('num_records:', len(recs))
    if recs:
        print('first_record_keys:', sorted(recs[0].keys()))

### Real run (optional)

To perform a full restoration run (non-dry), switch `--dry-run` off. This may download model weights on first use and take longer:

```python
from restoria.cli.main import main as restoria_main
img = 'assets/gfpgan_logo.png'
out_dir = 'results/real_demo/notebook_quickstart'
restoria_main(['run', '--input', img, '--output', out_dir, '--device', 'auto', '--metrics', 'fast'])
```

Tips:
- Use `--seed` and `--deterministic` for reproducibility
- Try `--compile` for potential speed-ups (falls back safely if unsupported)
- ONNX Runtime path requires the ORT extras installed; otherwise it’s skipped gracefully

In [ ]:
# 6) Optional: metrics demo (fast). Skip in CI to avoid heavy deps/weights.
import os, sys
if os.getenv('CI','0') != '1':
    from pathlib import Path
    if 'restoria' not in sys.modules:
        sys.path.insert(0, str(Path.cwd() / 'src'))
    from restoria.cli.main import main as restoria_main
    img = 'assets/gfpgan_logo.png' if os.path.exists('assets/gfpgan_logo.png') else 'tmp/blank.png'
    out_dir = 'results/metrics_demo'
    rc = restoria_main(['run','--input', img,'--output', out_dir,'--device','auto','--metrics','fast'])
    print('metrics demo rc:', rc)
    mp = os.path.join(out_dir,'metrics.json')
    if os.path.exists(mp):
        import json
        data = json.load(open(mp))
        ms = data.get('metrics', [])
        if ms and isinstance(ms, list):
            print('arcface_cosine:', ms[0].get('metrics',{}).get('arcface_cosine'))
else:
    print('CI detected; skipping metrics demo.')